<a href="https://colab.research.google.com/github/rashmi12305/credit-risk-model/blob/main/torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
dir(datasets)

['CIFAR10',
 'CIFAR100',
 'CLEVRClassification',
 'CREStereo',
 'Caltech101',
 'Caltech256',
 'CarlaStereo',
 'CelebA',
 'Cityscapes',
 'CocoCaptions',
 'CocoDetection',
 'Country211',
 'DTD',
 'DatasetFolder',
 'EMNIST',
 'ETH3DStereo',
 'EuroSAT',
 'FER2013',
 'FGVCAircraft',
 'FakeData',
 'FallingThingsStereo',
 'FashionMNIST',
 'Flickr30k',
 'Flickr8k',
 'Flowers102',
 'FlyingChairs',
 'FlyingThings3D',
 'Food101',
 'GTSRB',
 'HD1K',
 'HMDB51',
 'INaturalist',
 'ImageFolder',
 'ImageNet',
 'Imagenette',
 'InStereo2k',
 'KMNIST',
 'Kinetics',
 'Kitti',
 'Kitti2012Stereo',
 'Kitti2015Stereo',
 'KittiFlow',
 'LFWPairs',
 'LFWPeople',
 'LSUN',
 'LSUNClass',
 'MNIST',
 'Middlebury2014Stereo',
 'MovingMNIST',
 'Omniglot',
 'OxfordIIITPet',
 'PCAM',
 'PhotoTour',
 'Places365',
 'QMNIST',
 'RenderedSST2',
 'SBDataset',
 'SBU',
 'SEMEION',
 'STL10',
 'SUN397',
 'SVHN',
 'SceneFlowStereo',
 'Sintel',
 'SintelStereo',
 'StanfordCars',
 'UCF101',
 'USPS',
 'VOCDetection',
 'VOCSegmentation',
 

In [ ]:
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)
train_data

100%|██████████| 26421880/26421880 [00:07<00:00, 3584079.71it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 197815.13it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3675027.60it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21231344.14it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
train_loader = DataLoader(train_data, batch_size = 64)
test_loader = DataLoader(test_data, batch_size = 64)

In [ ]:
for X, y in train_loader:
  print(X.shape)
  print(y.shape)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [ ]:
device = "cuda"

In [ ]:
class NNfunction(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NNfunction().to(device)
print(model)

NNfunction(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    loss=loss_fn(model(X), y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  loss, correct = 0, 0
  for X, y in dataloader:
    X, y = X.to(device), y.to(device)
    pred=model(X)
    loss+= loss_fn(pred, y)
    correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  loss /= num_batches
  correct /= size
  print(loss)
  print(correct*100)

In [ ]:
epoch = 5
for t in range(epoch):
  print(f"Epoch {t+1}\n----------------------------------")
  train(train_loader, model, loss_fn, optimizer)
  test(test_loader, model, loss_fn)

Epoch 1
----------------------------------
loss: 2.293453 [    0/60000]
loss: 2.299369 [ 6400/60000]
loss: 2.296298 [12800/60000]
loss: 2.300451 [19200/60000]
loss: 2.290045 [25600/60000]
loss: 2.299598 [32000/60000]
loss: 2.300323 [38400/60000]
loss: 2.300098 [44800/60000]
loss: 2.300018 [51200/60000]
loss: 2.314382 [57600/60000]
tensor(2.2977, device='cuda:0', grad_fn=<DivBackward0>)
15.17
Epoch 2
----------------------------------
loss: 2.293453 [    0/60000]
loss: 2.299369 [ 6400/60000]
loss: 2.296298 [12800/60000]
loss: 2.300451 [19200/60000]
loss: 2.290045 [25600/60000]
loss: 2.299598 [32000/60000]
loss: 2.300323 [38400/60000]
loss: 2.300098 [44800/60000]
loss: 2.300018 [51200/60000]
loss: 2.314382 [57600/60000]
tensor(2.2977, device='cuda:0', grad_fn=<DivBackward0>)
15.17
Epoch 3
----------------------------------
loss: 2.293453 [    0/60000]
loss: 2.299369 [ 6400/60000]
loss: 2.296298 [12800/60000]
loss: 2.300451 [19200/60000]
loss: 2.290045 [25600/60000]
loss: 2.299598 [32000/